## Crops / Monetary / Deflator calculation

With this code what are we going to do is extract the data from a website and modelate it for every country in a global Scale. There are three tables to operate.
Production, Deflation and Monetary. Once we have finished with them, we do the final operation and export it as csv

In the future, we want to improve this code, by pointing to a table, like the lookup table, 

Data source: [FAO](http://www.fao.org/faostat/en/#data/QC)

In [1]:
#importing the libraries
import requests
import pandas as pd
import zipfile
import os

In [ ]:
#This part is for the parsing of the data inside. It did not work jet. We will pass it for now
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

url = 'Request URL: http://fenixservices.fao.org/faostat/api/v1/en/data/QC?area=11&area_cs=FAO&element=2312&item=515&item_cs=FAO&year=1972&show_codes=true&show_unit=true&show_flags=true&null_values=false&page_number=1&page_size=100&output_type=objects'

## Crops production

In [2]:
#Get the file parameters
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

#Make sure that the URL works, since it has changed one time
#old_url_production="http://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_E_All_Data.zip"

url_production = 'http://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data.zip'

In [3]:
#Open and download the file
os.chdir(r"C:\Users\ruben.crespo\Downloads") #C:\Users\ruben.crespo\Documents\03_tests\FAO
zip_file = requests.get(url_production, headers=headers)
with open("Production_Crops_Livestock_E_All_Data_NOFLAG.zip", "wb") as file:
    file.write(zip_file.content)
    
#print all the files  inside the zip to choose which one do we want
with zipfile.ZipFile("Production_Crops_Livestock_E_All_Data_NOFLAG.zip") as zip:
    zip.printdir()

File Name                                             Modified             Size
Production_Crops_Livestock_E_All_Data.csv      2021-05-26 12:37:24     77500682
Production_Crops_Livestock_E_All_Data_NOFLAG.csv 2021-05-26 12:37:34     60777983
Production_Crops_Livestock_E_Flags.csv         2021-05-26 12:50:10          245


In [14]:
#info Source: https://samukweku.github.io/data-wrangling-blog/python/pandas/compressed%20data/zip/archived%20data/2020/07/21/Extract-DataFrame-from-Compressed-Data-into-Pandas.html
#Open the table
with zipfile.ZipFile("Production_Crops_Livestock_E_All_Data_NOFLAG.zip") as zip:
    data = zip.open("Production_Crops_Livestock_E_All_Data_NOFLAG.csv") #open the zip file
df_p = pd.read_csv(data, encoding= "ISO-8859-1", header=0) #The encoding was giving me problems

#Transform the table
df_p = df_p.drop(["Area Code", "Item Code", "Element Code"], axis = 1) #axis 1 for columns 0 for rows
df_p = df_p[df_p['Element'].str.match('Production')] #take rows with the "production value" from the "Element" column
df_p = df_p.drop(["Element", "Unit"], axis = 1) #Delete columns

df_p = df_p.reset_index(drop=True)  #reset index and delete old index
df_p = df_p.rename(columns={'Item': ''}) #take out the Item part as is giving problems
df_p.set_index(['Area', ''], inplace=True) #we create from the dataframe a multiindex with these two columns
df_p.sort_index(inplace=True) #we sort the data
df_p = df_p.stack() #take the header of the production alements and make a column with it


df_p = df_p.unstack(1) #take the 2nd column and make the header 
# df_p = df_p.rename(columns={'Item': None})
df_p = df_p.reset_index(level=[0,1])    #multiindex to single index, indicate the columns
df_p['level_1'] = df_p['level_1'].map(lambda x: x.lstrip('Y')) #remove the letter Y from year
df_p.rename(columns={'level_1': 'Year'}, inplace=True) #rename column for year

df_p

"""filter the countries"""

Belgium_Luxembourg = (df_p['Area'] == 'Belgium-Luxembourg') & (df_p['Year'] < '2000') #create a filter and use it the the dataframe. En caso de que vaya lento, hacer un filtro más preciso
df_Belgium_Luxembourg = df_p.loc[Belgium_Luxembourg] #create copy dataframe out of the filter
df_Belgium_Luxembourg.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Belgium = df_Belgium_Luxembourg * 0.97 #we do the operatiion
df_Belgium = df_Belgium.reset_index(level=[0,1])  #we undo the multyindex
df_Belgium['Area'].replace({'Belgium-Luxembourg': 'Belgium'}, inplace=True)
df_p.append(df_Belgium, ignore_index=True)

df_Luxembourg = df_Belgium_Luxembourg * 0.03 #we do the operatiion
df_Luxembourg = df_Luxembourg.reset_index(level=[0,1])  #we undo the munltiindex
df_Luxembourg['Area'].replace({'Belgium-Luxembourg': 'Luxembourg'}, inplace=True)
df_p = df_p.append(df_Luxembourg, ignore_index=True)

df_p = df_p[df_p.Area != 'Belgium-Luxembourg']

Czechoslovakia = (df_p['Area'] == 'Czechoslovakia') & (df_p['Year'] < '1993') #create a filter and use it the the dataframe
df_Czechoslovakia = df_p.loc[Czechoslovakia] #create dataframe out of the filter
df_Czechoslovakia.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Czechia = df_Czechoslovakia * 0.69 #we do the operatiion
df_Czechia = df_Czechia.reset_index(level=[0,1])  #we undo the munltiindex
df_Czechia['Area'].replace({'Czechoslovakia': 'Czechia'}, inplace=True)
df_p.append(df_Czechia, ignore_index=True)

df_Slovakia = df_Czechoslovakia * 0.31 #we do the operatiion
df_Slovakia = df_Slovakia.reset_index(level=[0,1])  #we undo the munltiindex
df_Slovakia['Area'].replace({'Czechoslovakia': 'Slovakia'}, inplace=True)
df_p = df_p.append(df_Slovakia, ignore_index=True)

df_p = df_p[df_p.Area != 'Czechoslovakia']

Serbia_and_Montenegro = (df_p['Area'] == 'Serbia and Montenegro') & (df_p['Year'] < '2006') #create a filter and use it the the dataframe
df_Serbia_and_Montenegro = df_p.loc[Serbia_and_Montenegro] #create dataframe out of the filter
df_Serbia_and_Montenegro.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Serbia = df_Serbia_and_Montenegro * 0.94 #we do the operatiion
df_Serbia = df_Serbia.reset_index(level=[0,1])  #we undo the munltiindex
df_Serbia['Area'].replace({'Serbia and Montenegro': 'Serbia'}, inplace=True)
df_p = df_p.append(df_Serbia, ignore_index=True)

df_Montenegro = df_Serbia_and_Montenegro * 0.06 #we do the operatiion
df_Montenegro = df_Montenegro.reset_index(level=[0,1])  #we undo the munltiindex
df_Montenegro['Area'].replace({'Serbia and Montenegro': 'Montenegro'}, inplace=True)
df_p = df_p.append(df_Montenegro, ignore_index=True)

df_p = df_p[df_p.Area != 'Serbia and Montenegro']

Ethiopia_PDR = (df_p['Area'] == 'Ethiopia PDR') & (df_p['Year'] < '1993') #create a filter and use it the the dataframe
df_Ethiopia_PDR = df_p.loc[Ethiopia_PDR] #create dataframe out of the filter
df_Ethiopia_PDR.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Eritrea = df_Ethiopia_PDR * 0.04 #we do the operatiion
df_Eritrea = df_Eritrea.reset_index(level=[0,1])  #we undo the munltiindex
df_Eritrea['Area'].replace({'Ethiopia PDR': 'Eritrea'}, inplace=True)
df_p = df_p.append(df_Eritrea, ignore_index=True)

df_Ethiopia = df_Ethiopia_PDR * 0.96 #we do the operatiion
df_Ethiopia = df_Ethiopia.reset_index(level=[0,1])  #we undo the munltiindex
df_Ethiopia['Area'].replace({'Ethiopia PDR': 'Ethiopia'}, inplace=True)
df_p = df_p.append(df_Ethiopia, ignore_index=True)

df_p = df_p[df_p.Area != 'Ethiopia PDR']

Sudan_former = (df_p['Area'] == 'Sudan (former)') & (df_p['Year'] < '2011') #create a filter and use it the the dataframe
df_Sudan_former = df_p.loc[Sudan_former] #create dataframe out of the filter
df_Sudan_former.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Sudan = df_Sudan_former * 1 #we do the operatiion
df_Sudan = df_Sudan.reset_index(level=[0,1])  #we undo the munltiindex
df_Sudan['Area'].replace({'Sudan (former)': 'Sudan'}, inplace=True)
df_p = df_p.append(df_Sudan, ignore_index=True)

df_South_Sudan = df_Sudan_former * 0 #we do the operatiion
df_South_Sudan = df_South_Sudan.reset_index(level=[0,1])  #we undo the munltiindex
df_South_Sudan['Area'].replace({'Sudan (former)': 'South Sudan'}, inplace=True)
df_p = df_p.append(df_South_Sudan, ignore_index=True)

df_p = df_p[df_p.Area != 'Sudan (former)']

Indonesia = (df_p['Area'] == 'Indonesia') & (df_p['Year'] < '2011') #create a filter and use it the the dataframe
df_Indonesia_main = df_p.loc[Indonesia] #create dataframe out of the filter
df_Indonesia_main.set_index(['Area', 'Year'], inplace=True) #we create a multiindex to do the operation
df_Timor_Leste = df_Indonesia_main * 0.001 #we do the operatiion
df_Timor_Leste = df_Timor_Leste.reset_index(level=[0,1])  #we undo the munltiindex
df_Timor_Leste['Area'].replace({'Indonesia': 'Timor Leste'}, inplace=True)
df_p = df_p.append(df_Timor_Leste, ignore_index=True)


df_Indonesia = df_Indonesia_main * 0.999 #we do the operatiion
df_Indonesia = df_Indonesia.reset_index(level=[0,1])  #we undo the multyindex
df_p = df_p[df_p.Area != 'Indonesia'] #we delete the original Indonesia
df_p = df_p.append(df_Indonesia, ignore_index=True) #we import the new Indonesia


"""Prepare the data to operate"""
df_p = df_p.sort_values(by=['Area', 'Year'])
df_p = df_p.reset_index(drop=True)  #reset index and delete old index

#since we want to do all in python, we prepare the table for the final operation as a multyindex
df_p.set_index(['Area', 'Year'], inplace=True)
df_p

#the table is ready to be exported as csv (just in case)
df_p.to_csv('production_data_1.csv', index=False)

## Deflator

In [4]:
#Get the file parameters
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

url_deflator = 'http://fenixservices.fao.org/faostat/static/bulkdownloads/Deflators_E_All_Data.zip'

In [ ]:
#Open and download the file
os.chdir(r"C:\Users\ruben.crespo\Downloads") #C:\Users\ruben.crespo\Documents\03_tests\FAO
zip_file = requests.get(url_deflator, headers=headers)
with open("Deflators_E_All_Data.zip", "wb") as file:
    file.write(zip_file.content)

#print all the files  inside the zip to choose which one do we want
with zipfile.ZipFile("Deflators_E_All_Data.zip") as zip:
    zip.printdir()

In [17]:
os.chdir(r"C:\Users\ruben.crespo\Downloads") #C:\Users\ruben.crespo\Documents\03_tests\FAO
#Open the csv we want and see it
with zipfile.ZipFile("Deflators_E_All_Data.zip") as zip:
    deflator_data = zip.open("Deflators_E_All_Data_NOFLAG.csv") #open the zip file

df_d = pd.read_csv(deflator_data, encoding= "ISO-8859-1", header=0) #The encoding was giving me problems

# df_d.head()

#transform the table
df_d = df_d.drop(["Area Code", "Item Code", "Element Code", "Element"], axis = 1) #axis 1 for columns 0 for rows

filter_USD = (df_d['Item'] == 'GDP Deflator') & (df_d['Unit'] == 'US$') #create a filter and use it the the dataframe
df_d = df_d.loc[filter_USD]
df_d = df_d.drop(["Item", "Unit"], axis = 1) #Delete columns
df_d = df_d.reset_index(drop=True)  #reset index and delete old index
# df_r.sort_index(inplace=True) #we sort the data
df_d.columns = df_d.columns.str.replace('Y','') #we remove the Y letter


#we transform the table to operate in the final one
df_d.set_index(['Area'], inplace=True)
df_d = df_d.stack() #take the header and make a column with it
df_d = df_d.reset_index(level=[0,1])
df_d.rename(columns={'level_1': 'Year', 0:'deflator'}, inplace=True) #rename column for year


"""filter the countries"""
Czechoslovakia = (df_d['Area'] == 'Czechoslovakia') & (df_d['Year'] < '1990') #create a filter and use it the the dataframe
df_Czechoslovakia = df_d.loc[Czechoslovakia] #create dataframe out of the filter
df_Czechia = df_Czechoslovakia.copy() #we use the copy this time because we do no operation
df_Czechia['Area'].replace({'Czechoslovakia': 'Czechia'}, inplace=True)
df_d.append(df_Czechia, ignore_index=True)

df_Slovakia = df_Czechoslovakia.copy()
df_Slovakia['Area'].replace({'Czechoslovakia': 'SLovakia'}, inplace=True)
df_d.append(df_Slovakia, ignore_index=True)

df_d = df_d[df_d.Area != 'Czechoslovakia']

Ethiopia_PDR = (df_d['Area'] == 'Ethiopia PDR') & (df_d['Year'] < '1990') #create a filter and use it the the dataframe
df_Ethiopia_PDR = df_d.loc[Ethiopia_PDR] #create dataframe out of the filter
df_Eritrea = df_Ethiopia_PDR.copy()
df_Eritrea['Area'].replace({'Ethiopia PDR': 'Eritrea'}, inplace=True)
df_d.append(df_Eritrea, ignore_index=True)

df_Ethiopia = df_Ethiopia_PDR.copy()
df_Ethiopia['Area'].replace({'Ethiopia PDR': 'Ethiopia'}, inplace=True)
df_d.append(df_Ethiopia, ignore_index=True)

df_d = df_d[df_d.Area != 'Ethiopia PDR']

Sudan_former = (df_d['Area'] == 'Sudan (former)') & (df_d['Year'] < '2008') #create a filter and use it the the dataframe
df_Sudan_former = df_d.loc[Sudan_former] #create dataframe out of the filter
df_Sudan = df_Sudan_former.copy()
df_Sudan['Area'].replace({'Sudan (former)': 'Sudan'}, inplace=True)
df_d = df_d.append(df_Sudan, ignore_index=True)

df_South_Sudan = df_Sudan_former.copy()
df_South_Sudan['Area'].replace({'Sudan (former)': 'South Sudan'}, inplace=True)
df_d = df_d.append(df_South_Sudan, ignore_index=True)

df_d = df_d[df_d.Area != 'Sudan (former)']

Indonesia = (df_d['Area'] == 'Indonesia') & (df_d['Year'] < '2011') #create a filter and use it the the dataframe
df_Indonesia_main = df_d.loc[Indonesia] #create dataframe out of the filter
df_Timor_Leste = df_Indonesia_main.copy()
df_Timor_Leste['Area'].replace({'Indonesia': 'Timor Leste'}, inplace=True)
df_d = df_d.append(df_Timor_Leste, ignore_index=True)

df_Indonesia = df_Indonesia_main.copy()
df_d = df_d[df_d.Area != 'Indonesia'] #we delete the original Indonesia
df_d = df_d.append(df_Indonesia, ignore_index=True) #we import the new Indonesia

"""Prepare the data"""
df_d = df_d.sort_values(by=['Area', 'Year']) #sort the values
df_d = df_d.reset_index(drop=True)  #reset index and delete old index
df_d.set_index(['Area', 'Year'], inplace=True)
df_d_s = df_d.squeeze() #We transform the data from Dataframe to Series
df_d_s

#the table is ready to be exported as csv
df_d.to_csv('deflator_data.csv', index=False)

In [ ]:
#this was, to veryfy if there are duplicate values, if they were, there is something wrong going on at elaborating the tables
df_p_duplicated = df_p[df_p.index.duplicated()]
df_d_duplicated = df_p_duplicated.reset_index(level=[0,1])
df_d_duplicated

## Monetary Production

In [6]:
#Get the file parameters
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

url_monetary = 'http://fenixservices.fao.org/faostat/static/bulkdownloads/Prices_E_All_Data.zip'

In [47]:
#Open and download the file
os.chdir(r"C:\Users\ruben.crespo\Downloads") #C:\Users\ruben.crespo\Documents\03_tests\FAO
zip_file = requests.get(url_monetary, headers=headers)
with open("Prices_E_All_Data.zip", "wb") as file:
    file.write(zip_file.content)

#print all the files  inside the zip to choose which one do we want
with zipfile.ZipFile("Prices_E_All_Data.zip") as zip:
    zip.printdir()

File Name                                             Modified             Size
Prices_E_All_Data.csv                          2021-05-18 11:39:34     31763335
Prices_E_All_Data_NOFLAG.csv                   2021-05-18 11:39:38     26125513
Prices_E_Flags.csv                             2021-05-18 11:44:58          142


In [16]:
#info Source: https://samukweku.github.io/data-wrangling-blog/python/pandas/compressed%20data/zip/archived%20data/2020/07/21/Extract-DataFrame-from-Compressed-Data-into-Pandas.html
#Open and see the csv we want
with zipfile.ZipFile("Prices_E_All_Data.zip") as zip:
    monetary_data = zip.open("Prices_E_All_Data_NOFLAG.csv")
df_m = pd.read_csv(monetary_data, encoding= "ISO-8859-1", header=0) #The encoding was giving me problems

#Transform the table
df_m = df_m.drop(["Area Code", "Item Code", "Element Code", "Months Code"], axis = 1) #axis 1 for columns 0 for rows
filter_monetary = (df_m['Months'] == 'Annual value') & (df_m['Unit'] == 'USD') #create a filter and use it the the dataframe
df_m = df_m.loc[filter_monetary]

df_m = df_m.drop(["Element", "Months", "Unit"], axis = 1) #Delete unnecessary columns

df_m = df_m.reset_index(drop=True)  #reset index and delete old index
df_m = df_m.rename(columns={'Item': ''}) #take out the Item part as is giving problems
df_m.set_index(['Area', ''], inplace=True) #we create from the dataframe a multiindex with these two columns
df_m.sort_index(inplace=True) #we sort the data

df_m = df_m.stack() #take the header and make a column with it
df_m = df_m.unstack(1) #take the 2nd column and make the header 
df_m = df_m.reset_index(level=[0,1])    #multiindex to single index, indicate the columns
# df_m = df_m.drop(["Item"], axis = 1) #Delete Item columns
df_m['level_1'] = df_m['level_1'].map(lambda x: x.lstrip('Y')) #remove the letter Y from year
df_m.rename(columns={'level_1': 'Year'}, inplace=True) #rename column for year
# df_m.to_csv('monetary_data.csv', index=False)

"""filter the countries"""

Belgium_Luxembourg = (df_m['Area'] == 'Belgium-Luxembourg') & (df_m['Year'] < '2000') #create a filter and use it the the dataframe. En caso de que vaya lento, hacer un filtro más preciso
df_Belgium_Luxembourg = df_m.loc[Belgium_Luxembourg] #create copy dataframe out of the filter
df_Belgium = df_Belgium_Luxembourg.copy()
df_Belgium['Area'].replace({'Belgium-Luxembourg': 'Belgium'}, inplace=True)
df_p.append(df_Belgium, ignore_index=True)

df_Luxembourg = df_Belgium_Luxembourg.copy()
df_Luxembourg['Area'].replace({'Belgium-Luxembourg': 'Luxembourg'}, inplace=True)
df_m = df_m.append(df_Luxembourg, ignore_index=True)

df_m = df_m[df_m.Area != 'Belgium-Luxembourg']

Czechoslovakia = (df_m['Area'] == 'Czechoslovakia') & (df_m['Year'] == '1991') #create a filter and use it the the dataframe
df_Czechoslovakia = df_m.loc[Czechoslovakia] #create dataframe out of the filter
df_Czechia = df_Czechoslovakia.copy()
df_Czechia['Area'].replace({'Czechoslovakia': 'Czechia'}, inplace=True)
df_m.append(df_Czechia, ignore_index=True)

df_Slovakia = df_Czechoslovakia.copy()
df_Slovakia['Area'].replace({'Czechoslovakia': 'Slovakia'}, inplace=True)
df_m = df_m.append(df_Slovakia, ignore_index=True)

df_m = df_m[df_m.Area != 'Czechoslovakia']

Serbia_and_Montenegro = (df_m['Area'] == 'Serbia and Montenegro') & (df_m['Year'] < '2006') #create a filter and use it the the dataframe
df_Serbia_and_Montenegro = df_m.loc[Serbia_and_Montenegro] #create dataframe out of the filter
df_Serbia = df_Serbia_and_Montenegro.copy()
df_Serbia['Area'].replace({'Serbia and Montenegro': 'Serbia'}, inplace=True)
df_m = df_m.append(df_Serbia, ignore_index=True)

df_Montenegro = df_Serbia_and_Montenegro.copy()
df_Montenegro['Area'].replace({'Serbia and Montenegro': 'Montenegro'}, inplace=True)
df_m = df_m.append(df_Montenegro, ignore_index=True)

df_m = df_m[df_m.Area != 'Serbia and Montenegro']

Ethiopia_PDR = (df_m['Area'] == 'Ethiopia PDR') & (df_m['Year'] < '1993') #create a filter and use it the the dataframe
df_Ethiopia_PDR = df_m.loc[Ethiopia_PDR] #create dataframe out of the filter
df_Eritrea = df_Ethiopia_PDR.copy()
df_Eritrea['Area'].replace({'Ethiopia PDR': 'Eritrea'}, inplace=True)
df_m = df_m.append(df_Eritrea, ignore_index=True)

df_Ethiopia = df_Ethiopia_PDR.copy()
df_Ethiopia['Area'].replace({'Ethiopia PDR': 'Ethiopia'}, inplace=True)
df_m = df_m.append(df_Ethiopia, ignore_index=True)

df_m = df_m[df_m.Area != 'Ethiopia PDR']

Sudan_former = (df_m['Area'] == 'Sudan (former)') & (df_m['Year'] < '2012') #create a filter and use it the the dataframe
df_Sudan_former = df_m.loc[Sudan_former] #create dataframe out of the filter
df_Sudan = df_Sudan_former.copy()
df_Sudan['Area'].replace({'Sudan (former)': 'Sudan'}, inplace=True)
df_m = df_m.append(df_Sudan, ignore_index=True)

df_South_Sudan = df_Sudan_former.copy()
df_South_Sudan['Area'].replace({'Sudan (former)': 'South Sudan'}, inplace=True)
df_m = df_m.append(df_South_Sudan, ignore_index=True)

df_m = df_m[df_m.Area != 'Sudan (former)']

Indonesia = (df_m['Area'] == 'Indonesia') & (df_m['Year'] < '2020') #create a filter and use it the the dataframe
df_Indonesia_main = df_m.loc[Indonesia] #create dataframe out of the filter
df_Timor_Leste = df_Indonesia_main.copy()
df_Timor_Leste['Area'].replace({'Indonesia': 'Timor Leste'}, inplace=True)
df_m = df_m.append(df_Timor_Leste, ignore_index=True)

df_Indonesia = df_Indonesia_main.copy()
df_m = df_m[df_m.Area != 'Indonesia'] #we delete the original Indonesia
df_m = df_m.append(df_Indonesia, ignore_index=True) #we import the new Indonesia

"""Prepare the data"""

df_m = df_m.sort_values(by=['Area', 'Year'])
df_m = df_m.reset_index(drop=True)  #reset index and delete old index
#since we want to do all in python, we prepare the table for the final operation as a multyindex
df_m.set_index(['Area', 'Year'], inplace=True)
#the table is ready to be exported as csv
df_m.to_csv('monetary_data.csv', index=False)
df_m

Agave fibres nes  Almonds, with shell  \
Area     Year                                          
Albania  1993               NaN                  NaN   
         1994               NaN                  NaN   
         1995               NaN                  NaN   
         1996               NaN                  NaN   
         1997               NaN                  NaN   
...                         ...                  ...   
Zimbabwe 2014               NaN                  NaN   
         2015               NaN                  NaN   
         2016               NaN                  NaN   
         2017               NaN                  NaN   
         2018               NaN                  NaN   

               Anise, badian, fennel, coriander  Apples  Apricots  Areca nuts  \
Area     Year                                                                   
Albania  1993                               NaN   461.0       NaN         NaN   
         1994                               NaN     NaN       NaN         NaN   
         1995                               NaN   323.6       NaN         NaN   
         1996                               NaN   325.4       NaN         NaN   
         1997                               NaN   253.1       NaN         NaN   
...                                         ...     ...       ...         ...   
Zimbabwe 2014                               NaN     NaN       NaN         NaN   
         2015                               NaN  1897.0       NaN         NaN   
         2016                               NaN  1789.0       NaN         NaN   
         2017                               NaN  1865.0       NaN         NaN   
         2018                               NaN     NaN       NaN         NaN   

               Artichokes  Asparagus  Avocados  Bambara beans  ...  Vanilla  \
Area     Year                                                  ...            
Albania  1993         NaN        NaN       NaN            NaN  ...      NaN   
         1994         NaN        NaN       NaN            NaN  ...      NaN   
         1995         NaN        NaN       NaN            NaN  ...      NaN   
         1996         NaN        NaN       NaN            NaN  ...      NaN   
         1997         NaN        NaN       NaN            NaN  ...      NaN   
...                   ...        ...       ...            ...  ...      ...   
Zimbabwe 2014         NaN        NaN       NaN            NaN  ...      NaN   
         2015         NaN        NaN    1009.0            NaN  ...      NaN   
         2016         NaN        NaN    1007.0            NaN  ...      NaN   
         2017         NaN        NaN    1158.0            NaN  ...      NaN   
         2018         NaN        NaN       NaN            NaN  ...      NaN   

               Vegetables, fresh nes  Vegetables, leguminous nes  Vetches  \
Area     Year                                                               
Albania  1993                  437.1                         NaN      NaN   
         1994                  427.0                         NaN      NaN   
         1995                    NaN                         NaN      NaN   
         1996                    NaN                         NaN      NaN   
         1997                    NaN                         NaN      NaN   
...                              ...                         ...      ...   
Zimbabwe 2014                    NaN                         NaN      NaN   
         2015                  671.0                         NaN      NaN   
         2016                 1727.0                         NaN      NaN   
         2017                  607.0                         NaN      NaN   
         2018                    NaN                         NaN      NaN   

               Walnuts, with shell  Watermelons  Wheat  Wool, greasy  Yams  \
Area     Year                                                                
Albania  1993                  NaN          Na

## Calculations for the final table

In [13]:
#time to calculate
"""USD@2015 prices = LCU prices * 100 / LCU deflator@2015 /Exchange rate for USD in 2015"""

# df_usd = df_p * 100 / df_d / df_m #it does not work

df_usd = df_p * 100 / df_m #this works
#df_m_s is a series type data, we operate by rows
df_final = df_usd.div(df_d_s,  axis = "rows",  fill_value = None)


df_final
#We undo the multiindex (if we want)
# df_usd = df_usd.reset_index(drop=True)
# df_usd = df_usd.reset_index(level=[0,1])

Agave fibres nes  Almonds, with shell  \
Area        Year                                          
Afghanistan 1961               NaN                  NaN   
            1962               NaN                  NaN   
            1963               NaN                  NaN   
            1964               NaN                  NaN   
            1965               NaN                  NaN   
...                            ...                  ...   
Zimbabwe    2015               NaN                  NaN   
            2016               NaN                  NaN   
            2017               NaN                  NaN   
            2018               NaN                  NaN   
            2019               NaN                  NaN   

                  Anise, badian, fennel, coriander    Apples  Apricots  \
Area        Year                                                         
Afghanistan 1961                               NaN       NaN       NaN   
            1962                               NaN       NaN       NaN   
            1963                               NaN       NaN       NaN   
            1964                               NaN       NaN       NaN   
            1965                               NaN       NaN       NaN   
...                                            ...       ...       ...   
Zimbabwe    2015                               NaN  3.510807       NaN   
            2016                               NaN  3.583817       NaN   
            2017                               NaN  3.337341       NaN   
            2018                               NaN       NaN       NaN   
            2019                               NaN       NaN       NaN   

                  Areca nuts  Artichokes  Asparagus  Avocados  Bambara beans  \
Area        Year                                                               
Afghanistan 1961         NaN         NaN        NaN       NaN            NaN   
            1962         NaN         NaN        NaN       NaN            NaN   
            1963         NaN         NaN        NaN       NaN            NaN   
            1964         NaN         NaN        NaN       NaN            NaN   
            1965         NaN         NaN        NaN       NaN            NaN   
...                      ...         ...        ...       ...            ...   
Zimbabwe    2015         NaN         NaN        NaN  1.684836            NaN   
            2016         NaN         NaN        NaN  1.749677            NaN   
            2017         NaN         NaN        NaN  2.475394            NaN   
            2018         NaN         NaN        NaN       NaN            NaN   
            2019         NaN         NaN        NaN       NaN            NaN   

                  ...  Walnuts, with shell  Watermelons      Wheat  \
Area        Year  ...                                                
Afghanistan 1961  ...                  NaN          NaN        NaN   
            1962  ...                  NaN          NaN        NaN   
            1963  ...                  NaN          NaN        NaN   
            1964  ...                  NaN          NaN        NaN   
            1965  ...                  NaN          NaN        NaN   
...               ...                  ...          ...        ...   
Zimbabwe    2015  ...                  NaN          NaN  90.604000   
            2016  ...                  NaN          NaN  82.238819   
            2017  ...                  NaN          NaN  73.984540   
            2018  ...                  NaN          NaN  65.751426   
            2019  ...                  NaN          NaN        NaN   

                  Whey, condensed  Whey, dry  Wine  Wool, greasy  Yams  \
Area        Year                                                         
Afghanistan 1961              NaN        NaN   NaN           NaN   NaN   
            1962              NaN        NaN   NaN           NaN   NaN   
            1963              NaN      

In [37]:
df_p_list = list(df_p)
df_m_list = list(df_m)
#https://www.programiz.com/python-programming/methods/set/symmetric_difference
#these functions neen set type data imports
symetric_difference = list(set(df_p_list).symmetric_difference(set(df_m_list))) #these are the different values from both
df_p_list_difference = list(set(df_p_list).difference(set(df_m_list))) 
df_m_list_difference = list(set(df_m_list).difference(set(df_p_list)))
intersection = list(set(df_m_list).intersection(set(df_p_list)))
# df_m_list_difference = df_m_list.difference(df_p_list)

"""
total_p: 283
total_m: 212
equal: 190
different_total: 115
different production: 93
different monetary: 22
"""

## Other stuff, put it into the cheat sheet so we don't forget

In [ ]:
#Prepare the Year column as a Series data type
df_transposed = df.T #rows to columns
df_transposed[0] = df_transposed[0].map(lambda x: x.lstrip('Y')) #remove the letter Y
year = df_transposed[0].drop([0,1,2,3,4,5,6]) #drop first 6 rows, there are 59 years
all_years = [year] * 246 #number of countries in list
year_concat = pd.concat(all_years) #concat the years
year_series = year_concat.reset_index(drop=True)  #reset index and delete old index


In [ ]:
#Prepare the country column, It has regions bigger than country, but we will filter them at the end
countries = df[1].drop([0])
countries.head() #It is a Series
unique_countries = countries.unique() #is an array
#print(len(country.unique())) #246 unique elements
#print(len(country)) #51510 total elements
country_list = []
for country in unique_countries:
    country_multiplier = [country] * 59
    country_list.extend(country_multiplier)

country_series = pd.Series(country_list)


In [ ]:
#product header. We will create a list out of the product
product = df[3].drop([0])
print(len(product)) #175 unique elements
product_array = product.unique()
product_list = product_array.tolist()


New problem with the production, they mixed the Livestock with the crop production, so we have to filter out everything that is not crop. In the Element column, the "area harvested" value takes all the crop production values, we will take that

In [ ]:
#product crop header. We will filter the livestock and create a list out of the product
area_harvested = df[5]=='Area harvested'
crops_df = df[area_harvested]
product = crops_df[3]
print(len(product)) #175 unique elements
product_array = product.unique()
product_list = product_array.tolist()

In [ ]:
#final table
base_frame = { 'Country': country_series, 'Year': year_series }
product_frame = { i : None for i in product_list }

base_df = pd.DataFrame(base_frame) #these already have an index in the series
product_df = pd.DataFrame(product_frame, index=[0]) #use an index with scalar values
template_df = pd.concat([base_df, product_df], axis=1)

In [ ]:
#fill the table with the data of the original table
for row_index, row in template_df.loc[1:3].iterrows():
    for column_index[0:], value in row.items():
        print("this is the value: " + value)
        print(row_index)
        print("this is the column_index: " + column_index) #column_index es el valor de la cabecera de la columna
        print(row) # row es un Series de index por columna y valor de contenido
        
        # print(value)
    # for coklumn_index_base, value_base in row.items():

#time

# True = template_df["Year"] = df_transposed[0]
    

In [ ]:
for row_index, row in df.loc[0:3].iterrows():
    for column_index, value in row.items():
        print (value)

In [ ]:
#test
# df_row_value = df.loc[1="Afghanistan"][3="Apples"]
df_row_value_test = df.loc[1]
# df_time = df_row.iloc["Y" + "1967"]
print (df_row_value_test)

In [ ]:
#GID column. There is no GID data inside the csv, so we have to extract it from somewhere
GID_path = r"C:\Users\ruben.crespo\Documents\03_tests\FAO\ID_extended_gadm.csv"

GID_df = pd.read_csv(GID_path, encoding= "ISO-8859-1", header=None) #The encoding was giving me problems

GID_df.head()

